In [100]:
##### importing custom modules from the projects folder
import sys, os
from pathlib import Path
# Add project root to sys.path - search backwards through folders to find config.py
cwd = Path.cwd()
# Search upwards until a "config*" file is found
for parent in [cwd, *cwd.parents]:
    match = next(parent.glob('config*'), None)
    if match:
        PROJECT_ROOT = match.parent
        break
sys.path.append(str(PROJECT_ROOT))
import modules.helperModule as hf
##### -------------------------------------------------
from datetime import date
import pandas as pd

In [101]:
dfDrafts = hf.query_database(
    'SELECT * FROM ktbdrafts;'
)
dfProj = hf.query_database(
    """
    SELECT player.name, pos.pos, p.* 
    FROM projection p
    INNER JOIN player ON p.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    WHERE season = 2025
    """
)
dfRank = hf.query_database(
    """
    SELECT player.name, pos.pos, o.outletName, a.analystName, r.* 
    FROM ranking r
    INNER JOIN player ON r.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    LEFT JOIN outlet o ON r.outletId = o.outletId
    LEFT JOIN analyst a ON r.analystId = a.analystID
    WHERE season = 2025;
    """
)
dfAdp= hf.query_database(
    """
    SELECT player.name, pos.pos, o.outletName, a.* 
    FROM adp a
    INNER JOIN player ON a.playerId = player.playerId
    LEFT JOIN pos ON player.posId = pos.posId
    LEFT JOIN outlet o ON a.outletId = o.outletId
    WHERE date > '2025-04-30';
    """
)

dfPlayers = hf.query_database(
    """
    SELECT kp.playerId, kp.playerName, pos.pos, kp.adp, kp.points, kp.pointsAvg,  
    kp.points/kp.pointsAvg gp, kp.positionRank, kp.overallRank
    FROM ktbplayers kp
    LEFT JOIN player p ON kp.playerId = p.espnId 
    LEFT JOIN pos ON p.posId = pos.posId
    WHERE season = 2024 AND points > 0;
    """
).drop_duplicates(keep='first')

query successful
query successful
query successful
query successful
query successful


<head> VBD CALCS </head>

<ul>
<li>VOLS: replacement = first round after starters can be filled. 9-starter roster = 10th round, so 12 team league would be pick 121.</li>
<li>VORP: replacement = players not drafted so players ranked after draft</li>
<li>Man Games: </li>
<li>Pos by Starter: number of startes at position * number of teams. 1QB, 2RB, 3WR 12 team = QB replacement at 12th ranked, rb at 24th, wr at 36th</li>
<li>historical league drafting habits</li>
<li></li>


</ul>



In [104]:
# looking at my leagues historical drafting tendencies
# ====================================================
pick_cutoff = 120
season_cutoff = 2021
n_teams = 12
# needed starters is rosters spots in the league times teams in the league
needed_starters = {
    'WR': n_teams * 2,
    'RB': n_teams * 2,
    'QB': n_teams * 1,
    'TE': n_teams * 1,
    'FLEX': n_teams * 1
}

# determine the posistions tha will fill the flex spot.  highest overall ranks for 
# RB/WR/TE that were positional rank worse than the eneded starter
# WR has 2 slots, 12 teams = 24 needed starters. eligible wR to fill the flex would be 
# WR ranked > 24 
dfFlex = dfPlayers[
        (dfPlayers['pos']=='WR') & (dfPlayers['positionRank']>needed_starters['WR']) |
        (dfPlayers['pos']=='RB') & (dfPlayers['positionRank']>needed_starters['RB']) |
        (dfPlayers['pos']=='TE') & (dfPlayers['positionRank']>needed_starters['TE']) 
    ].sort_values('points', ascending=False).iloc[0:needed_starters['FLEX'],]
flex_starters = dfFlex['pos'].value_counts().to_dict()
del dfFlex

# based off last years stats, add the flex players needed starter dict
for k,v in flex_starters.items():
    needed_starters[k] += v

df_needed_starters = pd.DataFrame(
    list(needed_starters.items()), columns=['pos', 'needed_starters']
)

# ---- all years 
df_all = dfDrafts.query(f'overallPick <= {pick_cutoff}')
season_pos_counts = df_all.groupby(['season', 'pos']).size().reset_index(name='count')
avg_all = season_pos_counts.groupby('pos')['count'].mean().astype(int).reset_index()
avg_all = avg_all.rename(columns={'count': 'top_picks_all_years'})

# ---- last 3 years 
df_last3 = dfDrafts.query(f'overallPick <= {pick_cutoff} & season >= {season_cutoff}')
season_pos_counts = df_last3.groupby(['season', 'pos']).size().reset_index(name='count')
avg_last3 = season_pos_counts.groupby('pos')['count'].mean().astype(int).reset_index()
avg_last3 = avg_last3.rename(columns={'count': 'top_picks_last_3_years'})

# ---- merge 
avg_pos_counts = (
    avg_all.merge(avg_last3, on='pos', how='outer')
    .fillna(0)  # in case a pos is missing in one period
    .sort_values('top_picks_all_years', ascending=False)
    .reset_index(drop=True)
)

avg_pos_counts = (
    avg_pos_counts.merge(df_needed_starters, on='pos', how='outer')
    .fillna(0)  # in case a pos is missing in one period
    .sort_values('top_picks_all_years', ascending=False)
    .reset_index(drop=True)
)
pos_keep = ['QB', 'RB', 'WR', 'TE']
avg_pos_counts = avg_pos_counts[avg_pos_counts['pos'].isin(pos_keep)]


# lookup the replacement player score for the previous season. 
score_cols = avg_pos_counts.columns

replacement_player_score = pd.DataFrame({'pos':pos_keep})

# get the replacement scores for each calc style, skip first index since that will be pos label and not score index
for i in range(1,len(avg_pos_counts.columns)):
    #positions = []
    replacement_scores = []

    col_name = score_cols[i] + '_baseline_score'

    # loop through the positions
    for j in pos_keep: 
        #positions.append(j)
        
        
        replacement_rank = avg_pos_counts[(avg_pos_counts['pos']==j)].iloc[0,i]

        replacement_score = dfPlayers.loc[
            (dfPlayers['pos'] == j) & 
            (dfPlayers['positionRank'] == replacement_rank), 
            'points'
        ].iloc[0]

        replacement_scores.append(replacement_score)

        #replacement_player_score[i] = replacement_score


    temp = pd.DataFrame({col_name:replacement_scores})
    replacement_player_score = pd.concat([replacement_player_score, temp], axis=1)


avg_pos_counts = (
    avg_pos_counts.merge(replacement_player_score, on='pos', how='outer')
        .fillna(0)  # in case a pos is missing in one period
        .sort_values('top_picks_all_years', ascending=False)
        .reset_index(drop=True)
)
avg_pos_counts

,pos,top_picks_all_years,top_picks_last_3_years,needed_starters,top_picks_all_years_baseline_score,top_picks_last_3_years_baseline_score,needed_starters_baseline_score
0,WR,43.0,46.0,33.0,135.30,127.70,161.70
1,RB,40.0,38.0,27.0,110.10,110.60,162.20
2,QB,13.0,13.0,12.0,280.36,280.36,296.52
3,TE,11.0,11.0,12.0,120.50,120.50,116.10


In [106]:
import pandas as pd

# parameters
pick_cutoff = 120
season_cutoff = 2021
n_teams = 12
needed_starters = {'WR': 2, 'RB': 2, 'QB': 1, 'TE': 1, 'FLEX': 1}
needed_starters = {k: v * n_teams for k, v in needed_starters.items()}

# ---- flex logic
flex_eligible = dfPlayers.query(
    "(pos == 'WR' & positionRank > @needed_starters['WR']) | "
    "(pos == 'RB' & positionRank > @needed_starters['RB']) | "
    "(pos == 'TE' & positionRank > @needed_starters['TE'])"
)
flex_starters = (
    flex_eligible.nlargest(needed_starters['FLEX'], 'points')['pos']
    .value_counts()
    .to_dict()
)
for k, v in flex_starters.items():
    needed_starters[k] += v

df_needed_starters = pd.DataFrame(
    list(needed_starters.items()), columns=['pos', 'needed_starters']
)

# ---- helper to compute averages
def avg_counts(df, cutoff_query, col_name):
    counts = (
        df.query(cutoff_query)
        .groupby(['season', 'pos']).size()
        .groupby('pos').mean().astype(int)
        .rename(col_name)
    )
    return counts.reset_index()

avg_all = avg_counts(dfDrafts, f'overallPick <= {pick_cutoff}', 'top_picks_all_years')
avg_last3 = avg_counts(
    dfDrafts, f'overallPick <= {pick_cutoff} & season >= {season_cutoff}',
    'top_picks_last_3_years'
)

# ---- merge summary
pos_keep = ['QB', 'RB', 'WR', 'TE']
avg_pos_counts = (
    avg_all.merge(avg_last3, on='pos', how='outer')
    .merge(df_needed_starters, on='pos', how='outer')
    .fillna(0)
    .query('pos in @pos_keep')
    .sort_values('top_picks_all_years', ascending=False)
    .reset_index(drop=True)
)

# ---- replacement scores
def lookup_score(row, col):
    rank = int(row[col])
    match = dfPlayers.query("pos == @row['pos'] & positionRank == @rank")['points']
    return match.iloc[0] if not match.empty else 0

for col in avg_pos_counts.columns[1:]:
    avg_pos_counts[f'{col}_baseline_score'] = avg_pos_counts.apply(
        lambda r: lookup_score(r, col), axis=1
    )

avg_pos_counts


,pos,top_picks_all_years,top_picks_last_3_years,needed_starters,top_picks_all_years_baseline_score,top_picks_last_3_years_baseline_score,needed_starters_baseline_score
0,WR,43.0,46.0,33.0,135.30,127.70,161.70
1,RB,40.0,38.0,27.0,110.10,110.60,162.20
2,QB,13.0,13.0,12.0,280.36,280.36,296.52
3,TE,11.0,11.0,12.0,120.50,120.50,116.10


In [ ]:
dfPlayers[
        (dfPlayers['pos']=='TE')].sort_values('points', ascending=False).head(20)

In [ ]:
dfPlayers.sort_values('overallRank', ascending=True).head(50)

In [18]:
dfDrafts[(dfDrafts['season'] == 2024) & (dfDrafts['pos']=='QB')]

,teamName,season,pick,round,overallPick,name,playerTeam,pos,teamId
2471,DPD DannyDimes,2024,12,2,24,Lamar Jackson,Bal,QB,3.0
2473,The Suavin Scoregasms,2024,2,3,26,Josh Allen,Buf,QB,11.0
2481,Big Baby Nate,2024,10,3,34,Jalen Hurts,Phi,QB,9.0
2489,John's,2024,6,4,42,Patrick Mahomes,KC,QB,1.0
2493,Team Chaunce,2024,10,4,46,C.J. Stroud,Hou,QB,7.0
2504,Poopstained Warriors,2024,9,5,57,Dak Prescott,Dal,QB,4.0
2508,breeced lightning,2024,1,6,61,Anthony Richardson,Ind,QB,12.0
2515,JB got last in 23,2024,8,6,68,Joe Burrow,Cin,QB,5.0
2530,Team Gomer,2024,11,7,83,Kyler Murray,Ari,QB,2.0
2539,JB got last in 23,2024,8,8,92,Caleb Williams,Chi,QB,5.0
